In [78]:
# %matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import timeit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [91]:
classifiers = {
        "RBF SVM":              
            SVC(gamma=2, C=1),
        "Decision Tree":        
            DecisionTreeClassifier(max_depth=5),
        "Random Forest":        
            RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
        "Neural Net(Adam)":     
            MLPClassifier(alpha=1, max_iter=500, solver='adam'), # , verbose=10
        "Neural Net(Adam, a0.1)":     
            MLPClassifier(alpha=.1, max_iter=500, solver='adam'), # , verbose=10
        "Neural Net(Adam, a0.01)":     
            MLPClassifier(alpha=.01, max_iter=500, solver='adam'), # , verbose=10
        "Neural Net(Adam)":     
            MLPClassifier(alpha=1, max_iter=500, solver='adam'), # , verbose=10
        "Naive Bayes":          
            GaussianNB(),
        "Neural Net(Sgd)":      
            MLPClassifier(alpha=1, max_iter=500, solver='sgd'), # , verbose=10
        #"Linear SVM":           SVC(kernel="linear", C=0.025),
        "Nearest Neighbors":    
            KNeighborsClassifier(3),
        "Gaussian Process":     
            GaussianProcessClassifier(1.0 * RBF(1.0)),
        "AdaBoost":             
            AdaBoostClassifier(),
        #"QDA":                  QuadraticDiscriminantAnalysis(),
    }
display(classifiers)
hall0Pir = ('hall0Pir', 0)
hall0door = ('hall0door', 1)
cameraFront = ('cameraFront', 2)
cameraBack = ('cameraBack', 3)
corridor1Pir = ('corridor1Pir', 4)

{'AdaBoost': AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
           learning_rate=1.0, n_estimators=50, random_state=None),
 'Decision Tree': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best'),
 'Gaussian Process': GaussianProcessClassifier(copy_X_train=True,
              kernel=1**2 * RBF(length_scale=1), max_iter_predict=100,
              multi_class='one_vs_rest', n_jobs=1, n_restarts_optimizer=0,
              optimizer='fmin_l_bfgs_b', random_state=None,
              warm_start=False),
 'Naive Bayes': GaussianNB(priors=None),
 'Nearest Neighbors': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            metric_params=None, n_jobs=1, n_nei

In [71]:
%run home_events_poc.py
ctx = HomeEventsContext()

ctx.AddSequence('alert', {
    0 : corridor1Pir,
    1 : [hall0Pir, hall0door],
    9 : cameraFront
})

ctx.AddSequence('alert', {
    0 : corridor1Pir,
    1 : hall0Pir,
    9 : cameraBack
})

ctx.AddSequence('alert', {
    0 : corridor1Pir,
    1 : hall0Pir,
    8 : cameraBack,
    9 : cameraFront
})

ctx.AddSequence('master_come', {
    0 : corridor1Pir,
    5 : [hall0Pir, hall0door],
    6 : cameraFront
})

ctx.AddSequence('masterChild_come', {
    0 : corridor1Pir,
    5 : [hall0Pir, hall0door],
    9 : cameraFront
})

ctx.AddSequence('anomalie', {
    0 : corridor1Pir,
})

ctx.AddSequence('anomalie', {
    0 : hall0Pir,
})

#for i, n in zip(ctx.images_X, ctx.images_Y):
#    display(n, i)

In [94]:
X = np.asarray(ctx.images_X).reshape(len(ctx.images_Y), 60)
Y = np.asarray(ctx.images_Y)

X_pred1 = [[1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0]]
#X_pred = np.asarray(images_X[0]).reshape(1, 60)

for c in classifiers:

    clf = classifiers[c]
    start_time = timeit.default_timer()
    clf.fit(X, Y)
    elapsed = timeit.default_timer() - start_time
    pred_Y = clf.predict(X)
    selfscore = clf.score(X, Y)
    matched = [i for i, j in zip(Y, pred_Y) if i != j]
    if len(matched) != 0:
        #display('### ' + c + ' => ' + ('%.2f' % selfscore).lstrip('0') + ' (' + elapsed + ' sec)')
        display('### {:30} => {:.2f} ({:.2f} sec)'.format(c, selfscore, elapsed))
        #display(len(matched))
        #display(pred_Y)
    else:
        display('### {:30} => OK {:.2f} ({:.2f} sec)'.format(c, selfscore, elapsed))


'### RBF SVM                        => OK 1.00 (0.00 sec)'

'### Decision Tree                  => OK 1.00 (0.00 sec)'

'### Random Forest                  => OK 1.00 (0.06 sec)'

'### Neural Net(Adam)               => OK 1.00 (1.09 sec)'

'### Neural Net(Adam, a0.1)         => OK 1.00 (1.29 sec)'

...

C:\Prog\Anaconda\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


'### Neural Net(Sgd)                => OK 1.00 (1.14 sec)'

'### Nearest Neighbors              => 0.71 (0.00 sec)'

'### Gaussian Process               => 0.86 (0.46 sec)'

'### AdaBoost                       => 0.86 (0.24 sec)'

In [82]:
#X = StandardScaler().fit_transform(X)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4)

'0.86'

In [ ]:
# from real : lila & sergey with child
"""
21/01 ? Camera
21/01 ? Camera
21/01 12:58:01	Hall(0) Door	1	On	Door
21/01 12:58:07	Hall(0) Ceiling	1	On	Motion
21/01 12:58:24	Hall(0) Door	1	On	Door
21/01 12:59:32	Corridor(1) Ceiling	1	On	Motion
"""
# from real : sergey from work
"""
18/01 ? Camera
18/01 19:30:40	Hall(0) Door	1	On	Door
18/01 19:31:09	Hall(0) Ceiling	1	On	Motion
18/01 19:32:05	Corridor(1) Ceiling	1	On	Motion
"""
# from real : lilia with child
"""
18/01 19:01:53	Hall(0) Door	1	On	Door
18/01 19:02:40	Hall(0) Ceiling	1	On	Motion
18/01 19:05:50	Hall(0) Door	1	On	Door
18/01 19:06:19	Hall(0) Door	1	On	Door
18/01 19:07:04	Hall(0) Door	1	On	Door
18/01 19:08:47	Corridor(1) Ceiling	1	On	Motion
"""